# DeepChannel model implementation
This model was created for reasearch of Ion Channels @ University of Silesia in Katowice
Feel free to use it however you like

### Check if Tensorflow is installed and test for available GPU

In [9]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

tf.config.list_physical_devices()

TensorFlow version: 2.18.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Check if CuPy is available
CuPy is a subset of NumPy that can run on a CUDA gpu instead of CPU for added performance

In [10]:
import numpy as np
try:
    import cupy
    if cupy.cuda.is_available():
        np = cupy
        print("CUDA acceleration is available.")
    else:
        raise ValueError("CUDA accelerated GPU not found. Rolling back to CPU usage.")
except ValueError as e:
    print(e)

CUDA acceleration is available.


In [ ]:
# Define the Deep-Channel model
def build_deep_channel_model(input_shape, max_classes=5):
    """
    Build the Deep-Channel model.
    
    Parameters:
    - input_shape: tuple, the shape of the input data (time_steps, features).
    - max_classes: int, the maximum number of simultaneous open channels (default=5).
    
    Returns:
    - model: tf.keras.Model, the constructed Deep-Channel model.
    """
    inputs = tf.keras.Input(shape=input_shape)

    # 1D-CNN layers
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    
    # Flatten before feeding to LSTMs
    x = tf.keras.layers.Flatten()(x)
    
    # LSTM layers with dropout to prevent overfitting
    x = tf.keras.layers.Reshape((-1, input_shape[1]))(x)  # Reshape for LSTM
    x = tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.2)(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.2)(x)
    x = tf.keras.layers.LSTM(256, dropout=0.2)(x)
    
    # Dense layer with softmax activation for classification
    outputs = tf.keras.layers.Dense(max_classes + 1, activation='softmax')(x)

    # Build and compile the model
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Generate synthetic data using CuPy
def generate_synthetic_data(samples, time_steps, features, classes):
    """
    Generate synthetic training data.
    
    Parameters:
    - samples: int, the number of samples to generate.
    - time_steps: int, the length of each time series.
    - features: int, the number of features in the time series.
    - classes: int, the number of output classes.
    
    Returns:
    - x_data: np.ndarray, synthetic input data.
    - y_data: np.ndarray, corresponding labels.
    """
    x_data = np.random.randn(samples, time_steps, features).astype(np.float32)
    y_data = np.random.randint(0, classes, size=(samples,), dtype=np.int32)
    return x_data, y_data

# Parameters
time_steps = 1000   # Number of time steps in each sample
features = 1        # Number of features (e.g., current intensity)
classes = 5         # Max number of simultaneous open channels
samples = 1000      # Number of samples in the dataset

# Generate synthetic data
x_train, y_train = generate_synthetic_data(samples, time_steps, features, classes)

# Build and train the model
input_shape = (time_steps, features)
model = build_deep_channel_model(input_shape, max_classes=classes)

# Train the model
model.fit(
    x_train.get(), y_train.get(),  # Convert CuPy arrays to NumPy
    batch_size=2,
    epochs=10,
    validation_split=0.2
)

# Save the model
model.save("deep_channel_model.h5")


Epoch 1/10


2024-11-19 12:24:40.948871: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1048576000 exceeds 10% of free system memory.
2024-11-19 12:24:52.557702: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.88GiB (rounded to 5242880256)requested by op StatefulPartitionedCall/functional_1_1/lstm_3_1/CudnnRNNV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-11-19 12:24:52.557797: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1053] BFCAllocator dump for GPU_0_bfc
2024-11-19 12:24:52.557821: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (256): 	Total Chunks: 86, Chunks in use: 83. 21.5KiB allocated for chunks. 20.8KiB in use in bin. 2.5KiB client-requested in use in bin.
2024-11-19 12:24:52.557837: I exte

InternalError: Graph execution error:

Detected at node functional_1_1/lstm_3_1/CudnnRNNV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/jakubkraus/micromamba/envs/AIonChannel/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/jakubkraus/micromamba/envs/AIonChannel/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/jakubkraus/micromamba/envs/AIonChannel/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/jakubkraus/micromamba/envs/AIonChannel/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/jakubkraus/micromamba/envs/AIonChannel/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/home/jakubkraus/micromamba/envs/AIonChannel/lib/python3.12/asyncio/base_events.py", line 1986, in _run_o